<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/carseats.png?raw=true" width=800 align="left"/>

# 실습 내용

- 다양한 알고리즘으로 모델을 만들고 성능을 예측합니다.
- 성능이 좋을 것으로 판단된 모델의 성능을 튜닝합니다.
- 튜닝된 모델의 성능을 평가합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format = 'retina'

In [2]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/Carseats.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [50]:
# 데이터 살펴보기
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


**데이터 설명**

- Sales: 각 지역 판매량(단위: 1,000개)
- CompPrice: 경쟁사 가격 (단위: 달러)
- Income: 지역 평균 소득 (단위: 1,000달러)
- Advertising: 각 지역, 회사의 광고 예산 (단위: 1,000달러)
- Population: 지역 인구 수 (단위: 1,000명)
- Price: 자사 지역별 판매 가격 (단위: 달러)
- ShelveLoc: 진열 상태
- Age: 지역 인구의 평균 연령
- Education: 각 지역 교육 수준
- Urban: 도심 지역 여부 (Yes,No)
- US: 매장이 미국에 있는지 여부 (Yes, No)

In [4]:
# 기초통계정보 확인
data.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [5]:
# target 확인
target = 'Sales'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**2) 가변수화**

In [6]:
# 가변수화 대상: ShelveLoc, Education, Urban, US
dumm_cols = ['ShelveLoc', 'Education', 'Urban', 'US']

# 가변수화
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True)

# 확인
x.head()

,CompPrice,Income,Advertising,Population,Price,Age,ShelveLoc_Good,ShelveLoc_Medium,Education_11,Education_12,Education_13,Education_14,Education_15,Education_16,Education_17,Education_18,Urban_Yes,US_Yes
0,138,73,11,276,120,42,0,0,0,0,0,0,0,0,1,0,1,1
1,111,48,16,260,83,65,1,0,0,0,0,0,0,0,0,0,1,1
2,113,35,10,269,80,59,0,1,0,1,0,0,0,0,0,0,1,1
3,117,100,4,466,97,55,0,1,0,0,0,1,0,0,0,0,1,1
4,141,64,3,340,128,38,0,0,0,0,1,0,0,0,0,0,1,0


**3) 정규화**

In [7]:
# 정규화
x = (x - x.min()) / (x.max() - x.min())

# 확인
x.head()

,CompPrice,Income,Advertising,Population,Price,Age,ShelveLoc_Good,ShelveLoc_Medium,Education_11,Education_12,Education_13,Education_14,Education_15,Education_16,Education_17,Education_18,Urban_Yes,US_Yes
0,0.622449,0.525253,0.379310,0.533066,0.574850,0.309091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1,0.346939,0.272727,0.551724,0.501002,0.353293,0.727273,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.367347,0.141414,0.344828,0.519038,0.335329,0.618182,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,0.408163,0.797980,0.137931,0.913828,0.437126,0.545455,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,0.653061,0.434343,0.103448,0.661323,0.622754,0.236364,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


**4) 학습용, 평가용 데이터 분리**

In [8]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)  

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 다양한 알고리즘으로 모델을 만들고 성능을 확입니다.
- 성능 평가는 mean_absolute_error, r2_score를 사용합니다.

In [ ]:
# xgboost 설치
# pip install xgboost

In [ ]:
# lightgbm 설치
# pip install lightgbm

- 사용할 알고리즘을 모두 불러옵니다.

In [52]:
# 라이브러리 불러오기
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

**1) Linear Regression**

- Linear Regression 알고리즘으로 모델링하고 성능을 확인합니다.

In [10]:
# 선언하기
lr = LinearRegression()

In [11]:
# 학습하기
lr.fit(x_train,y_train)

LinearRegression()

In [12]:
# 예측하기
y_lr = lr.predict(x_test)

In [13]:
# 5단계: 평가하기
print(mean_absolute_error(y_test,y_lr))
print(r2_score(y_test,y_lr))

0.8291702856537123
0.8752579913475846


In [51]:
# 일반화 성능
cv_score = cross_val_score(lr,x_train,y_train,cv=5)
print(cv_score.mean())

0.8430257461788264


**2) KNN**

- KNN 알고리즘으로 모델링하고 성능을 확인합니다.

In [62]:
# 선언하기
kn = KNeighborsRegressor()

In [63]:
# 학습하기
kn.fit(x_train,y_train)

KNeighborsRegressor()

In [64]:
# 예측하기
y_kn = kn.predict(x_test)

In [65]:
# 평가하기
print(mean_absolute_error(y_test,y_kn))
print(r2_score(y_test,y_kn))

2.0890166666666667
0.2661301923711501


In [66]:
# 하이퍼파라미터 조정 모델 선언
params = {'n_neighbors':range(1,51)}
model_kn = GridSearchCV(kn,
                        params,
                        cv=5,
                        scoring='r2')

In [67]:
# 학습 
model_kn.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 100)}, scoring='r2')

In [68]:
# 최적 파라미터
model_kn.best_params_

{'n_neighbors': 23}

In [69]:
# 예측
y_model_kn = model_kn.predict(x_test)

In [70]:
# 평가하기
print(mean_absolute_error(y_test,y_model_kn))
print(r2_score(y_test,y_model_kn))

2.091623188405797
0.26546126248844615


**3) Decision Tree**

- Decision Tree 알고리즘으로 모델링하고 성능을 확인합니다.

In [20]:
# 선언하기
dt = DecisionTreeRegressor(max_depth=5,random_state=2022)

In [21]:
# 학습하기
dt.fit(x_train,y_train)

DecisionTreeRegressor(max_depth=5, random_state=2022)

In [22]:
# 예측하기
y_dt = dt.predict(x_test)

In [24]:
# 평가하기
print(mean_absolute_error(y_test,y_dt))
print(r2_score(y_test,y_dt))

1.7843856410406043
0.4615391298852002


In [ ]:
# 하이퍼파라미터 조정 모델 생성
params = {'max_depth':range(1,51)}
model_dt = 

**4) Random Forest**

- Random Forest 알고리즘으로 모델링하고 성능을 확인합니다.

In [27]:
# 선언하기
rf = RandomForestRegressor(max_depth=5,random_state=2022)

In [28]:
# 학습하기
rf.fit(x_train,y_train)

RandomForestRegressor(max_depth=5, random_state=2022)

In [29]:
# 예측하기
y_rf = rf.predict(x_test)

In [30]:
# 평가하기
print(mean_absolute_error(y_test,y_rf))
print(r2_score(y_test,y_rf))

1.4603246005172819
0.6294602394029838


**5) XGBoost**

- XGBoost 알고리즘으로 모델링하고 성능을 확인합니다.

In [33]:
# 선언하기
xgb = XGBRegressor(max_depth=5,random_state=2022)

In [34]:
# 학습하기
xgb.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=2022,
             reg_alpha=0, reg_lambda=1, ...)

In [37]:
# 예측하기
y_xgb = xgb.predict(x_test)

In [38]:
# 평가하기
print(mean_absolute_error(y_test,y_xgb))
print(r2_score(y_test,y_xgb))

1.3253447668552398
0.7107408087970767


**6) LightGBM**

- LightGBM 알고리즘으로 모델링하고 성능을 확인합니다.

In [41]:
# 선언하기
lgbm = LGBMRegressor(max_depth=5,random_state=2022)

In [42]:
# 학습하기
lgbm.fit(x_train,y_train)

LGBMRegressor(max_depth=5, random_state=2022)

In [43]:
# 예측하기
y_lgbm = lgbm.predict(x_test)

In [44]:
# 평가하기
print(mean_absolute_error(y_test,y_lgbm))
print(r2_score(y_test,y_lgbm))

1.1197044539176935
0.788778853305421
